# Build Training sets and Signature Alphabets

In [1]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"
dataset_directory = "C:/Users/meyerp/Documents/INRAE/Datasets"

import os
import time

os.chdir(path_directory)

import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

from signature.Signature import MoleculeSignature
from signature.signature_old import sanitize_molecule
from signature.signature_alphabet import load_alphabet, SignatureAlphabet, signature_from_smiles
from signature.utils import read_csv, read_tsv, read_txt, write_csv

In [2]:
use_smarts = True

## Build molecule datasets: sanitize molecules

In [ ]:
# Load file and sanitize molecule
# Final file format is two columns ID + canonical SMILES
# Warning: this cell is slow

def sanitize(data, MaxMolecularWeight, size):
    # Remove molecules with weight > MaxMolecularWeight
    # and with more than one piece. Make sure all molecules
    # are unique.
    D, SMI = [], set()
    for i in range(data.shape[0]):
        ID, smi = data[i, 0], str(data[i, 1])
        if smi == "nan":
            continue
        if i % 100000 == 0:
            print(f"-------{i} {data[i,0]} {data[i,1]} {len(smi)}")
        if smi.find(".") != -1:
            continue  # not in one piece
        if smi in SMI:
            continue  # aready there
        if len(smi) > int(MaxMolecularWeight / 5):  # Cheap skip
            continue
        mol, smi = sanitize_molecule(Chem.MolFromSmiles(smi))
        if mol == None:
            continue
        mw = Chem.Descriptors.ExactMolWt(mol)
        if mw > MaxMolecularWeight:
            continue
        if smi in SMI:
            continue  # canonical smi aready there
        SMI.add(smi)
        data[i, 1] = smi
        D.append(data[i])
        if len(D) >= size:
            break
    return np.asarray(D)


MaxMolecularWeight = 500
size = float("inf")
filename = dataset_directory + "/MetaNetX"
H, D = read_tsv(filename)
H = ["ID", "SMILES"]
D = D[:, [0, 8]]
np.random.shuffle(D)
print(f"size={D.shape[0]}")
D = sanitize(D, MaxMolecularWeight, size)
filename = f"{filename}_weight_{str(MaxMolecularWeight)}"
print(f"File={filename} Header={H} D={D.shape}")
#write_csv(filename, H, D)
"""
MaxMolecularWeight = 500
size = 1e6
filename = dataset_directory + "/emolecule"
H = ['ID', 'SMILES']
T = read_txt(filename+'.txt')
print(f'Size={len(T)}')
D = {}
for i in range(len(T)-1):
    line = list(T[i+1].split(' '))
    D[i] = [line[1]+'_'+line[2], line[0]]
D = np.asarray(list(D.values()))
np.random.shuffle(D)
print(f'File={filename} Header={H} D={D.shape}')
D = sanitize(D, MaxMolecularWeight, size)
filename = f'{filename}_weight_{str(MaxMolecularWeight)}'
print(f'File={filename} Header={H} D={D.shape}')
write_csv(filename, H, D)
"""

## Build training sets: filter molecules for generative models and deterministic enumeration

In [ ]:
# Compute signature in various format
def filter(smi, radius, verbose=False):
    if "." in smi:  #
        return "", "", None, ""
    if "*" in smi:  #
        return "", "", None, ""
    if "[" in smi:  # cannot process [*] without kekularization
        if "@" not in smi:
            return "", "", None, ""
    #Alphabet = SignatureAlphabet(radius=radius, nBits=0)
    #sig1, mol, smi = signature_from_smiles(smi, Alphabet, neighbor=True, verbose=False)
    #Alphabet = SignatureAlphabet(radius=radius, nBits=2048)
    #sig2, mol, smi = signature_from_smiles(smi, Alphabet, neighbor=True, verbose=False)
    
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return "", "", None, ""
    ms = MoleculeSignature(mol, radius=radius, neighbor=True, use_smarts=use_smarts, nbits=0)
    sig1 = ms.as_deprecated_string(morgan=False, neighbors=True)
    ms = MoleculeSignature(mol, radius=radius, neighbor=True, use_smarts=use_smarts, nbits=2048)
    sig2 = ms.as_deprecated_string(morgan=False, neighbors=True)
    
    if sig1 == "" or sig2 == "":
        return "", "", None, ""
    return sig1, sig2, mol, smi


# parameters
radius = 2
size = 1000
ext = f"_radius_{radius}_size_{size}"
if use_smarts:
    ext = ext + "_smarts"
# filename = './datasets/emolecule_weight_500'
filename = dataset_directory + "/MetaNetX_weight_500"
seed = 10
np.random.seed(seed=seed)

# Load Smiles file
H, D = read_csv(filename)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 1])))
print(f"Number of smiles: {len(Smiles)}")
np.random.shuffle(Smiles)

# Get to business
H = [
    "SMILES",
    "SIG",
    "SIG-NBIT",
]
D, i, I = {}, 0, 0
while True:
    sig1, sig2, mol, smi = filter(Smiles[i], radius)
    if sig1 == "" or sig2 == "":
        print(Smiles[i])
        i += 1
        continue
    D[I] = [smi, sig1, sig2]
    i, I = i + 1, I + 1
    if I == size:
        break
D = np.asarray(list(D.values()))
#write_csv(filename + ext, H, D)

## Build Signature Alphabet

In [3]:
# Compute signature in various format

# Parameters
radius = 2
nBits = 2048
allHsExplicit = False
ext = "_Alphabet"
ext += "_hydrogen" if allHsExplicit else ""
ext += f"_radius_{radius}_nBits_{nBits}"
if use_smarts:
    ext = ext + "_smarts"
file_smiles = dataset_directory + "/MetaNetX_weight_500"
file_alphabet = file_smiles + ext + "_full"
seed = 10
np.random.seed(seed=seed)

# Load Smiles file
H, D = read_csv(file_smiles)
print(f"Header={H}\nD={D.shape}")
Smiles = np.asarray(list(D[:, 1]))
#Smiles = Smiles[:1000, ]
print(f"Number of smiles: {len(Smiles)}")

# Create Alphabet
print("nBits", nBits)
Alphabet = SignatureAlphabet(radius=radius, nBits=nBits, use_smarts=use_smarts, boundary_bonds=False, map_root=True, legacy=False)

# Get save and load Alphabet
start_time = time.time()
Alphabet.fill(Smiles, verbose=False)
#Alphabet.save(file_alphabet)
#Alphabet = load_alphabet(file_alphabet)
ft = time.time() - start_time
print(f"CPU time compute Alphabet: {ft:.2f}")
Alphabet.print_out()

Header=['ID', 'SMILES']
D=(188547, 2)
Number of smiles: 188547
nBits 2048
... processing alphabet iteration: 0 size: 0 time: 0.000000
... processing alphabet iteration: 1000 size: 7566 time: 21.446350
... processing alphabet iteration: 2000 size: 12303 time: 25.213997
... processing alphabet iteration: 3000 size: 16355 time: 25.309272
... processing alphabet iteration: 4000 size: 19949 time: 26.151677
... processing alphabet iteration: 5000 size: 23138 time: 26.203394
... processing alphabet iteration: 6000 size: 26098 time: 25.644507
... processing alphabet iteration: 7000 size: 28868 time: 26.153434
... processing alphabet iteration: 8000 size: 31416 time: 25.916720
... processing alphabet iteration: 9000 size: 33777 time: 25.773216
... processing alphabet iteration: 10000 size: 35946 time: 34.652237
... processing alphabet iteration: 11000 size: 38155 time: 26.837553
... processing alphabet iteration: 12000 size: 40259 time: 34.211260
... processing alphabet iteration: 13000 size: 4

... processing alphabet iteration: 117000 size: 140121 time: 27.982087
... processing alphabet iteration: 118000 size: 140692 time: 27.166359
... processing alphabet iteration: 119000 size: 141252 time: 28.010385
... processing alphabet iteration: 120000 size: 141848 time: 28.306566
... processing alphabet iteration: 121000 size: 142436 time: 27.669123
... processing alphabet iteration: 122000 size: 142995 time: 27.028254
... processing alphabet iteration: 123000 size: 143646 time: 27.750412
... processing alphabet iteration: 124000 size: 144205 time: 28.177256
... processing alphabet iteration: 125000 size: 144778 time: 28.047100
... processing alphabet iteration: 126000 size: 145284 time: 28.035941
... processing alphabet iteration: 127000 size: 145877 time: 27.048079
... processing alphabet iteration: 128000 size: 146356 time: 26.671665
... processing alphabet iteration: 129000 size: 146967 time: 28.239795
... processing alphabet iteration: 130000 size: 147554 time: 27.767846
... pr

In [4]:
Alphabet.save(file_alphabet)